# DG/HDG splitting Methods (Unit 3.4)

When solving unsteady problems with an operator-splitting method it might be beneficial to consider different space discretizations for different operators.

For a problem of the form

$$\partial_t u + A u + C u = 0$$

We consider the operator splitting:

1. Step: Given $u_0$, solve $t^n \rightarrow t^{n+1}: \partial_t u + C u = 0 \implies u^{\frac{1}{2}}$
2. Step: Given $u^{\frac{1}{2}}$, solve $t^n\rightarrow t^{n+1}: \partial_t u + A u = 0 \rightarrow u^1$

This splitting is only first order accurate but allows to take different time discretizations for the substeps 1 and 2.

In this example we consider the Navier-Stokes problem again (cf. 3.2) and want to combine

- an $H(div)$-conforming Hybrid DG method (which is a very good discretization for Stokes-type problems) and 
- a standard upwind DG method for the discretization of the convection

The weak form is: Find $(u,p):[0,T]\times \Omega \rightarrow (H_{0,D}^1)^d\times L_2$, s.t.
$$\int_\Omega \partial_t u \cdot v + \int_\Omega \nu \nabla u : \nabla v + u \cdot \nabla u v - \int_\Omega \text{div}(v)p = \int fv\hspace{2em}\forall v \in (H_{0,D}^1)^d$$
$$- \int_\Omega \text{div}(u) q = 0 \hspace{2em}  \forall q \in L_2,$$
$$u(t=0) = u_0$$

Note: The original tutorial version just has $\int_\Omega \nu \nabla u \nabla v$ for the second term.  But that would be a product of matrices, hence a matrix.  All the other terms in the sum are clearly vectors so I must assume the Frobenius inner product is intended here.  Also, it's used in the formulation below.

Again, we consider the benchmark setup from http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html . The geometry is a 2D channel with a circular obstacle which is positioned (only slightly) off the center of the channel. The geometry: 
<img src="resources/geometry.png" \>

The left part (red) is the inflow boundary, The right part (green) is the outflow boundary. The viscosity is set to $\nu=10^{−3}$
.

In [1]:
import netgen.gui
%gui tk
import tkinter
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet") )
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain = 0, rightdomain = 1, bc = "cyl" )
mesh = Mesh( geo.GenerateMesh(maxh = 0.08) )
order = 3
mesh.Curve(order)

For the HDG formulation we use the product space with

- $BDM_k: H(div)$ conforming FE space (degree k)  What does BDM stand for?  Brezzi-Douglas-Marini
- Vector facet space: facet functions of degree k (vector valued and only in tangential direction)
- piecewise polynomials up to degree k−1 for the pressure 
<img src="resources/stokeshdghdiv.png" width='250px' \>
## HDG spaces

In [2]:
V1 = HDiv ( mesh, order = order, dirichlet = "wall|cyl|inlet" )
V2 = FESpace ( "vectorfacet", mesh, order = order, dirichlet = "wall|cyl|inlet" )
Q = L2( mesh, order = order-1)
V = FESpace ([V1,V2,Q])

u, uhat, p = V.TrialFunction()
v, vhat, q = V.TestFunction()

In [3]:
print(u.dims) # Remember HDiv elements are vectors with the same dimension as the mesh
gradu = CoefficientFunction( (grad(u),), dims=(2,2))
help(gradu.trans)

0: 2

Help on CoefficientFunction in module ngsolve.fem object:

class CoefficientFunction(pybind11_builtins.pybind11_object)
 |  A CoefficientFunction (CF) is some function defined on a mesh.
 |  Examples are coordinates x, y, z, domain-wise constants, solution-fields, ...
 |  CFs can be combined by mathematical operations (+,-,sin(), ...) to form new CFs
 |  Parameters:
 |  
 |  val : can be one of the following:
 |  
 |    scalar (float or complex):
 |      Creates a constant CoefficientFunction with value val
 |  
 |    tuple of scalars or CoefficientFunctions:
 |      Creates a vector or matrix valued CoefficientFunction, use dims=(h,w)
 |      for matrix valued CF
 |    list of scalars or CoefficientFunctions:
 |      Creates a domain-wise CF, use with generator expressions and mesh.GetMaterials()
 |      and mesh.GetBoundaries()
 |  
 |  Method resolution order:
 |      CoefficientFunction
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined

## Stokes discretization / initial conditions

The bilinear form to the HDG discretized Stokes problem is:

$$K_h((u_T, u_F, p),(v_T, v_F, q)) := \sum_{T \in \mathcal{T}}\left\lbrace \int_T \nu \nabla u_T : \nabla v_T dx
- \int_{\partial_T} \nu \frac{\partial u_T}{\partial n}\cdot[v]_t ds
- \int_{\partial_T} \nu \frac{\partial v_T}{\partial n}\cdot[u]_t ds
+ \int_{\partial_T} \nu \frac{\lambda k^2}{h}[u]_t\cdot [v]_t ds \right\rbrace$$
$$ - \int_\Omega \text{div}(u) q dx -  \int_\Omega \text{div}(v) p dx$$

where $[\cdot]_t$ is the tangential projection of the jump $(\cdot)_T−(\cdot)_F$.  (Here I think subscripts $T$ indicate elements and $F$ facets)

The mass matrix is simply
$$M_h((u_T, u_F, p),(v_T, v_F, q)) :=\int_\Omega u_T \cdot v_T dx$$


### Note:
The Discontinuous Galerkin Methods tutorial (2.8) uses a similar formulation.  In that tutorial, the whole thing (all 5 terms) represents an interior penalty DG form for $-\Delta u$.  I can't quite connect these yet.

I'm not sure yet why we use the $K_h$ notation (it seems to be referred to as 'a' below) or why we're combining the divergence terms into it.

I think the mass term is not present in the original equation at the top - i.e. the original equation above is only describing the Stokes part and doesn't include the convection (mass matrix term).

I'm not sure why we need the transposes below.  Assuming that $\nabla u$ is the row-wise gradient, it would seem that $\nabla u \cdot n$ would be just ```gradu * n```, a column vector. 

I think the only place we actually *need* InnerProduct below is the first line where we need Frobenius (confirmed).

I added explicit dot products to the second, third and fourth terms above, because that's the only interpretation that makes sense to me.

I think $\lambda$ is a sort of Lagrange multiplier.  It's renamed to alpha in the code below because ```lambda``` is a python keyword.

Note that HDiv elements are vectors with the same dimension as the mesh.  The test function u and its associated gridfunction simply represent velocity.

Notice the tricky use of CoefficientFunction below, passing it a tuple with one element which is grad(u) (where u is a vector), then specifying that we want it represented as a 2x2 matrix.
InnerProduct must be Frobenius.  Compare with the Navier-Stokes (3.2) which uses $H^1 \times H^1$ to represent $u$


In [4]:
nu = 0.001
alpha = 4

gradu = CoefficientFunction( (grad(u),), dims=(2,2) )
gradv = CoefficientFunction( (grad(v),), dims=(2,2) )

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(vec):
    return vec - (vec*n)*n

a = BilinearForm( V, symmetric=True)
a += SymbolicBFI( nu*InnerProduct ( gradu, gradv) )
a += SymbolicBFI( nu*InnerProduct ( gradu.trans * n,  tang(vhat-v) ), element_boundary=True )
a += SymbolicBFI( nu*InnerProduct ( gradv.trans * n,  tang(uhat-u) ), element_boundary=True )
a += SymbolicBFI( nu*alpha*order*order/h * InnerProduct ( tang(vhat-v),  tang(uhat-u) ), element_boundary=True )
a += SymbolicBFI( -div(u)*q -div(v)*p )
a.Assemble()

m = BilinearForm(V , symmetric=True)
m += SymbolicBFI( u * v )
m.Assemble()

f = LinearForm ( V )

As initial condition we solve the Stokes problem:

In [5]:
gfu = GridFunction(V)

U0 = 1.5
uin = CoefficientFunction( (U0*4*y*(0.41-y)/(0.41*0.41),0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))


invstokes = a.mat.Inverse(V.FreeDofs(), inverse="umfpack")

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += invstokes * res

Draw( gfu.components[0], mesh, "velocity" )
Draw( Norm(gfu.components[0]), mesh, "absvel(hdiv)")

## Application of convection

In the operator split approach we want to apply only operator applications for the convection part. Further, we want to do this in a usual DG setting. As a model problem we use the following procedure:

- Given $(u,p)$ in HDG space: project into $\hat{u}$ in usual DG space
- Solve $\partial_t \hat{u} = C\hat{u}$ by explicit scheme (involves convection evaluations and mass matrix operations only)
- Solve Unsteady Stokes step with r.h.s. from convection sub-problem. To this end evaluate mixed mass matrix $\int \hat{u}\cdot v$ to obtain a functional on the HDG space

For the projection steps we use mixed mass matrices:

- $M_m: HDG\times DG\rightarrow \mathbb{R}$
- $M_m^T: DG\times HDG\rightarrow \mathbb{R}$
- $M_{DG}: DG\times DG\rightarrow \mathbb{R}$ (block diagonal)

### mixed mass matrices

To set up mixed mass matrices we use a bilinear form with two different FESpaces.

We do not assemble the matrices as we will only need the matrix-vector applications of $M_m$, $M_m^T$ and $M_{DG}^{-1}$.

In [6]:
VL2 = L2(mesh, dim=mesh.dim, order=order, dgjumps=True )
uL2 = VL2.TrialFunction()   # 1 x dim matrix
vL2 = VL2.TestFunction()    

gfuL2 = GridFunction(VL2)

bfmixed = BilinearForm ( V, VL2, nonassemble=True )
bfmixed += SymbolicBFI ( u*vL2 )

bfmixedT = BilinearForm ( VL2, V, nonassemble=True )
bfmixedT += SymbolicBFI ( uL2*v )



### Notes:
We're using an overload of BilinearForm which takes a trial space and a test space, where V is still
```V = FESpace ([V1,V2,Q])``` (with dimension 3).  It's surprising that we can define a bilinear form from spaces with different dimensions.  bfmixed makes sense, though, because u and vL2 both have dimension 2  

The nonassemble flag prevents any memory from being allocated for a matrix.

The trial function uL2 is a 1 x dim matrix, because we specified dim=mesh.dim in the constructor of the space.

In previous tutorials, we had to jump through some hoops to compute grad(u) if u was a vector, but in this case it just works.  See next cell.

### convection operator

For the convection operation we use a standard Upwind scheme. Again, we do not set up the matrix as we are only interested in operator applications. For the advection velocity we use the $H(div)$-conforming velocity (which is point-wise divergence free).

In [7]:
vel = gfu.components[0]
convL2 = BilinearForm(VL2, nonassemble=True )
convL2 += SymbolicBFI( (-InnerProduct(grad(vL2).trans * vel, uL2.trans)) )
un = InnerProduct(vel,n)
upwindL2 = IfPos(un, un*uL2, un*uL2.Other(bnd=uin))
convL2 += SymbolicBFI( InnerProduct (upwindL2, vL2-vL2.Other()), VOL, skeleton=True )
convL2 += SymbolicBFI( InnerProduct (upwindL2, vL2), BND, skeleton=True )

### Notes:
Why are we specifying bnd=uin for uL2.Other?  Is the inlet the 'upwind' side?  I think passing the optional bnd argument to Other() restricts this to the specified boundary element.  

Skeleton=True specifies integrals on facets.  I think the use of VOL and BND indicates that the first integrator should integrate over the Volumes of the facets and the second should integrate over their boundaries

### solution of convection steps

We now define the solution of the convection problem for an initial data $u$ in the HDG space. The return value ("res") is $M_m \hat{u}$ where $\hat{u}$ is the solution of several explicit Euler steps of the convection problem
$$\partial_t \hat{u} = -M_{DG}^{-1} C\hat{u}$$ 

In [8]:
def SolveConvectionSteps(gfuvec, res, tau, steps):
    bfmixed.Apply (gfuvec, gfuL2.vec) 
    VL2.SolveM(vec=gfuL2.vec, rho=CoefficientFunction(1))
    conv_applied = gfuL2.vec.CreateVector()
    for i in range(steps):
        convL2.Apply(gfuL2.vec,conv_applied)
        VL2.SolveM(vec=conv_applied, rho=CoefficientFunction(1))
        gfuL2.vec.data -= tau/steps * conv_applied
        #Redraw()    
    bfmixedT.Apply (gfuL2.vec, res)
    
#SolveConvectionSteps(gfu,res,0.01,1)    
#Draw(gfuL2, mesh, "velocity(L2)")
#Draw(Norm(gfuL2), mesh, "absvel(L2)")



### Notes
Solve with the mass matrix.  Available only for L2-like spaces.
 I don't really understand how this works - we take a vector and a coefficient function.  what are we solving? 
 Is the solution stored and returned in the vector? Yes

bilinearform.Apply(x,y) Applies a (non-)linear variational formulation to x and stores the result in y.
 
To solve with the block-diagonal (or even diagonal) mass matrix of an *L2* -finite element space, we can use the *SolveM* method of the FESpace. The *rho* argument allows to specify a density coefficientfunction for the mass-matrix. The operation is performed inplace for the given vector.

Digging into the source code.  Each L2 element has a little mass matrix associated with it, not necessarily square, represented as a flat vector.  We take the vector passed to solveM (vec) get some kind of indirect representation of it as a matrix, and update the matrix with values obtained (roughly speaking) by integrating, applying the coefficient function, etc.

I don't think this has anything to do with the bilinear form 'mass matrix' defined above.

## Operator splitting method

In [9]:
# initial values again:
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += invstokes * res
t = 0
tend = 0

In [10]:
tend += 1
substeps = 10
tau = 0.01

mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + tau * a.mat.AsVector()
inv = mstar.Inverse(V.FreeDofs(), inverse="umfpack")

while t < tend:
    SolveConvectionSteps(gfu.vec, res, tau, substeps)
    res.data -= mstar * gfu.vec
    gfu.vec.data += inv * res
    t += tau
    print ("t=", t)
    Redraw(blocking=True) # blocking=True blocks until the Redraw completes so we can see each 'frame' of animation

t= 0.01
t= 0.02
t= 0.03
t= 0.04
t= 0.05
t= 0.060000000000000005
t= 0.07
t= 0.08
t= 0.09
t= 0.09999999999999999
t= 0.10999999999999999
t= 0.11999999999999998
t= 0.12999999999999998
t= 0.13999999999999999
t= 0.15
t= 0.16
t= 0.17
t= 0.18000000000000002
t= 0.19000000000000003
t= 0.20000000000000004
t= 0.21000000000000005
t= 0.22000000000000006
t= 0.23000000000000007
t= 0.24000000000000007
t= 0.25000000000000006
t= 0.26000000000000006
t= 0.2700000000000001
t= 0.2800000000000001
t= 0.2900000000000001
t= 0.3000000000000001
t= 0.3100000000000001
t= 0.3200000000000001
t= 0.3300000000000001
t= 0.34000000000000014
t= 0.35000000000000014
t= 0.36000000000000015
t= 0.37000000000000016
t= 0.38000000000000017
t= 0.3900000000000002
t= 0.4000000000000002
t= 0.4100000000000002
t= 0.4200000000000002
t= 0.4300000000000002
t= 0.4400000000000002
t= 0.45000000000000023
t= 0.46000000000000024
t= 0.47000000000000025
t= 0.48000000000000026
t= 0.49000000000000027
t= 0.5000000000000002
t= 0.5100000000000002
t= 0.5